# Artificial Intelligence Course, Dr.Fadaei

# Computer Assignment #1

## Parnian fazel (810198516)

### **Goal**

In this assignment, we are to find a solution for “Dr Strange” problem. To do this, we will use BFS and IDS as an Uninformed Search algorithm and A* as an Informed Search algorithm. by doing this project, we learn about mentioned search algotirhms and go through some details about them. Also, we will learn the advantages and disadvantages of each algorithm.

### **Brief Description**
In this project there is a doctor who should collect all potions in a grid. At the begining, one doctor is in the down left corner of the field. Beside the potions, there are some boost medicines which by using them, one doctor will be added to the field (the new doctor will begin moving at up right corner of the grid). If a doctor pass a sqaure that contain a potion or a boost medicine, it will be consumed by them and there is no choice for doctors. doctors can move in four durection in each move: up, down, left or right. The final goal is to collect all potions and doctors should reach the top right corner of the grid.

### Defining and modeling the problem:
* **initial state**: where the agent starts. Here is left corner of the map.
* **goal state**:  the state which all the potions have been eaten by the doctors and all the doctors are in the [n-1,m-1] sqaure (top right corner of the map).
* **actions**: there are four actions can be made: going up, down, left or right if the square is not wall or obstacle. pay attention when we are in a sqaure which has a potion or a boost medicine the doctors will use them and there is no choice, so using the potions and boost medicines does not count as an action
* **step cost**: the step cost in this project is always one
* **optimal solution**: the optimal solution here is the lowest path cost among all solutions is to get the top right corner of the map, eaten all the potions.

First I import the libraries I need in this project:

In [713]:
import queue
import copy
import collections
import time
import math
import heapq
import bisect

## Reading the test data and coordinatioans

I save the coordinations in a list ([x,y]).
* "potions" list contains list of potions coordinations.
* "boosts" list contains list of boosts medicine coordinations.
* "obs" list contains list of obstacles coordinations.

In [714]:
f = open("test3.in", "r")
n, m = f.readline().split()
c, k = f.readline().split()
n, m, c, k = int(n), int(m), int(c), int(k)

potions = []
boosts = []
obs = []

for i in range(c):
    x = f.readline().split()
    potions.append([int(x[0]), int(x[1])])
    
for i in range(k):
    x = f.readline().split()
    boosts.append([int(x[0]), int(x[1])])

    
obs_n = int(f.readline())
for i in range(obs_n):
    x = f.readline().split()
    obs.append([int(x[0]), int(x[1])])

I defined two classes in this project. One for agent and another one for state. The Agent class contains the x and y coordinations and also I assigned an unique ID to each agent because of the fact that there can be several agents in the field and the should be distinguishable.

In [715]:
class Agent:
    def __init__(self, x, y, a_id):
        self.x = x
        self.y = y
        self.id = a_id
    def to_string(self):
        return str(self.x) + '-' + str(self.y)

The State class has several attributes:
* **agents** -> agents cooridinations in each state
* **potions** -> potions remianed in each state
* **boosts** -> boosts medicines remained in each state
* **parent** -> parent of this state
* **depth** -> depth of each state from the root state
* **move** -> a string to save the direction of the moving agent.

In [716]:
class State:
    def __init__(self, agent, potions_coor, boost_coor, parent = None):
        self.agents = []
        self.agents.append(agent)
        self.potions = potions_coor
        self.boosts = boost_coor
        self.move = ''
        self.parent = parent
        if parent == None:
            self.depth = 0
        else: 
            self.depth = parent.depth + 1
        
    def add_agent(self, sec_agent):
        self.agents.append(sec_agent)
        
    def did_all_agents_reach_dest(self):
        x = 0
        for ag in self.agents:
            if ag.x == n-1 and ag.y == m-1:
                x+=1
        if x == len(self.agents):
            return True
        return False
    def is_goal(self):
        if self.did_all_agents_reach_dest() and not len(self.potions):
            return True
        return False
    def set_move(self, dir_move):
        self.move = dir_move
    def to_string(self): 
        res = ''
        for ag in self.agents:
            res += ag.to_string()
        for pot in self.potions:
            res += str(pot[0])+'-'+str(pot[1])
        for boo in self.boosts:
            res += str(boo[0])+'-'+str(boo[1])
        
        return res


Here I define the allowed directions each doctor can go and also some functions we need through the entire project.

In [717]:
directions = [[1, 0], [0, 1], [-1, 0], [0, -1]]

def is_obs(x, y):
    if x < 0 or y < 0 or y >= m or x >= n or [x, y] in obs: 
        return True
    return False


def is_potion_in_sqaure(curr_state, x, y):
    if [x, y] in curr_state.potions:
        return True
    return False

def is_boost_in_sqaure(curr_state, x, y):
    if [x, y] in curr_state.boosts:
        return True
    return False

def get_dir_string(dir):
    if(dir == [1, 0]): return "UP"
    if(dir == [-1, 0]): return "DOWN"
    if(dir == [0, 1]): return "RIGHT"
    if(dir == [0, -1]): return "LEFT"
    
def print_path(st):
    if st.parent == None:
        return 
    print_path(st.parent)
    print(f"{st.move}")

Here I define the initial state for the 3 search algorithms:

In [718]:
first_agent = Agent(0,0,1)
initial_state = State(first_agent, potions, boosts, None)
if is_boost_in_sqaure(initial_state, 0, 0):
    initial_state.boosts = [x for x in initial_state.boosts if x not in [[0, 0]]]
    new_agent_in = Agent(n-1, 0, 2)
    initial_state.add_agent(new_agent_in)

# **Uninformed search**

# Breadth-first Search

BFS is a strategy in which the root node is expanded first, then all the successors of the root node are expanded next, then their successors, and so on. Generally, all the nodes are expanded at a given depth in the search tree before any nodes at the next level are expanded.

### Measuring `BFS` performance:
* **Completeness**: Yes (if branching factor b is finite)
* **Optimality**: Yes, If the path cost is a nondecreasing function of the depth of the node
* **Time Complexity**: $b^{d}$ (b is branching factor and d is depth of the optimal solution
* **Space Complexity**: $b^{d}$

The memory requirements are a bigger problem for `BFS` than is the executation time.
The number of all states seen is equal to the number of unique states seen, because of the fact that this algorithm doesn't explore a state more than once.

I defined the BFS function as it has a frontier queue (FIFO queue) to store the list of unexpanded nodes and an explored set that every time we expand a node, we add its state.to-string() into this set. The reason I used the string of the state is beacause of the executation time. At first I append the initial state to the frontier list then a loop continues until we don't reach the goal node or the frontier set is non-empty. At each step we enqueue a node from frontier. the we check if the successor is the goal state, if it is the while should end the solution be returned and if not I expand the current state and add the expnaded state to the frontier set. I add state.to_string() into explored set beacuae in this case the executation time will decreas.

**Advantages**:

* The algorithm is complete (The solution will definitely be found by BFS If there is some solution).
* If there are more than one solution in the problem, it will find the shallowest solution.


**Disadvantages**:
* Memory Constraints As it stores all the nodes of the present level to go for the next level.
* If a solution is far away then it consumes time.

In [719]:
def BFS(initial_state):
    explored = set()
    frontier = collections.deque()
    frontier.append(initial_state)
    explored.add(initial_state.to_string())
    number_of_all_states = 1
    if initial_state.is_goal():
        print("GOAL REACHED")
        return initial_state, number_of_all_states, len(explored)
    while len(frontier):
        curr_state = frontier.popleft()
        for d in directions:
            for a in curr_state.agents:
                new_x = a.x + d[0]
                new_y = a.y + d[1]
                if not is_obs(new_x, new_y):
                    new_pos_ag = Agent(new_x, new_y, a.id)
                    new_state_potions = copy.deepcopy(curr_state.potions)
                    new_state_boosts = copy.deepcopy(curr_state.boosts)
                    if is_potion_in_sqaure(curr_state, new_x, new_y):
                        new_state_potions = [x for x in new_state_potions if x not in [[new_x, new_y]]]
                    boost_ag = None
                    if is_boost_in_sqaure(curr_state, new_x, new_y):
                        new_state_boosts = [x for x in new_state_boosts if x not in [[new_x, new_y]]]
                        boost_ag = Agent(n-1, 0, len(curr_state.agents)+1)
                    new_state = State(new_pos_ag, new_state_potions, new_state_boosts, curr_state)
                    number_of_all_states += 1
                    for other_ag in curr_state.agents:
                        if not other_ag.id == new_pos_ag.id:
                            new_state.add_agent(other_ag)
                    move_str = "(" + str(new_pos_ag.x)+","+str(new_pos_ag.y)+") "+str(new_pos_ag.id)+' '+get_dir_string(d)
                    if not boost_ag == None:
                        new_state.add_agent(boost_ag)
                        move_str += " *boost* number of agents: " + str(len(new_state.agents))
                    new_state.set_move(move_str)
                    if not new_state.to_string() in explored:
                        if new_state.is_goal():
                            print("GOAL REACHED")
                            return new_state, number_of_all_states, len(explored)
                        else:
                            frontier.append(new_state)
                            explored.add(new_state.to_string())

    print("FAILIUR")

Running the `BFS` algorithm:

In [720]:
start = time.time()
final_state, number_of_all_states, different_states = BFS(initial_state)
end = time.time()

GOAL REACHED


Printing the result of running `BFS` algorithm:

In [721]:
print(f"Distance: {final_state.depth}")
print("Optimal path:")
print_path(final_state)
print(f"Number of all states seen: {number_of_all_states}")
print(f"Number of different states seen: {different_states}")
print(f"Executation time: {end-start} second")

Distance: 19
Optimal path:
(1,0) 1 UP
(2,0) 1 UP
(2,1) 1 RIGHT
(2,2) 1 RIGHT *boost* number of agents: 2
(5,1) 2 RIGHT
(5,2) 2 RIGHT
(5,3) 2 RIGHT
(4,3) 2 DOWN
(5,3) 2 UP
(5,4) 2 RIGHT
(5,5) 2 RIGHT
(1,2) 1 DOWN
(1,3) 1 RIGHT
(1,4) 1 RIGHT
(1,5) 1 RIGHT
(2,5) 1 UP
(3,5) 1 UP
(4,5) 1 UP
(5,5) 1 UP
Number of all states seen: 1082141
Number of different states seen: 179473
Executation time: 16.8428373336792 second


## Test result of `BFS` search:

| Test# | Different states |  All states | Distance  |             time           |
|------ |----------------|-------------|-----------|----------------------------|
|   1   |      805       |    4149     |     11    |     0.05605125427246094    |
|   2   |     854407     |    116268   |     11    |     10.436302661895752     |
|   3   |     179473     |    1082141  |     19    |     15.891164064407349     |


# Iterartive deepening depth-first search

`IDS` is a general strategy, often used in combination with `depth-first search` that findes the best depth limit. It does this by gradually icreasing the limit until a goal is found. Iterative deepening combines the benefits of the `depth-first` and `breadth-first` search. Like `breadth-first search`, it is completewhen the branching factor is finite and optimal when the path cost is a non-decreasing function of the depth of the node.

### Measuring `IDS` performance:
* **Completeness**: Yes (if branching factor b is finite)
* **Optimality**: Yes, if step cost = 1
* **Time Complexity**: $(d+1)*b^{0} + d*b{1} + (d-1)*b^{2} + ... + b^{d} = O(b^{d})$
* **Space Complexity**: $b*d$

*`b` is branching factor and `d` is depth of the optimal solution

**Advantages**:
* `IDDFS` is complete if there is a solution.
* Another major advantage of the `IDDFS` algorithm is its quick responsiveness. The early results indications are a plus point in this algorithm.
* The performance of `IDDFS` is better than single `BFS` and `DFS` operating exclusively.
* This `IDS` is an optimal search algotrithm.

**Disadvantages**:
* The time taken is exponential to reach the goal node.
* The main problem with `IDDFS` is the time and wasted calculations that take place at each depth.


First I implemented `DLS` algorithm. the DLS algorithm runs the `DFS` algorithm with a limit of depth. For `IDS`, I call `DLS` in a while loop by incresing the depth limit until the goal is found. At every itteration I reset all of data structures to initial values. in `IDS` we should consider the depth of states in explored set, because it is possible to see a repeated state but in less depth.

In [722]:
def DLS(curr_state, limit, explored):
    global number_of_all_states
    explored.add((curr_state.to_string(), curr_state.depth))
    if curr_state.is_goal():
        print(f"Number of different states seen: {len(explored)}")
        print(f"Number of all states seen: {number_of_all_states}")
        return curr_state
    if curr_state.depth == limit:
        return None
    for d in directions:
        for a in curr_state.agents:
            new_x = a.x + d[0]
            new_y = a.y + d[1]
            if not is_obs(new_x, new_y):
                new_pos_ag = Agent(new_x, new_y, a.id)
                new_state_potions = copy.deepcopy(curr_state.potions)
                new_state_boosts = copy.deepcopy(curr_state.boosts)
                if is_potion_in_sqaure(curr_state, new_x, new_y):
                    new_state_potions = [x for x in new_state_potions if x not in [[new_x, new_y]]]
                new_ag = None
                if is_boost_in_sqaure(curr_state, new_x, new_y):
                    new_state_boosts = [x for x in new_state_boosts if x not in [[new_x, new_y]]]
                    new_ag = Agent(n-1, 0, len(curr_state.agents)+1)
                new_state = State(new_pos_ag, new_state_potions, new_state_boosts, curr_state)
                number_of_all_states += 1
                if not new_ag == None:
                    new_state.add_agent(new_ag)
                for other_ag in curr_state.agents:
                    if not other_ag.id == new_pos_ag.id:
                        new_state.add_agent(other_ag)
                move_str = "(" + str(new_pos_ag.x)+","+str(new_pos_ag.y)+") "+str(new_pos_ag.id)+' '+get_dir_string(d)
                if not new_ag == None:
                    move_str += " *boost* number of agents: " + str(len(new_state.agents))
                new_state.set_move(move_str)                
                if (new_state.to_string(), new_state.depth) not in explored:
                    result = DLS(new_state, limit, explored)
                    if not result == None:
                        return result


In [723]:
def IDS(initial_state):
    depth = 0
    explored = set()
    while True:
        explored.clear()
        final_state = DLS(initial_state, depth, explored)
        if not final_state == None:
            break
        depth += 1
    return final_state

In [724]:
number_of_all_states = 0
start = time.time()
final_state = IDS(initial_state)
end = time.time()
print("Optimal path:")
print_path(final_state)
print(f"Distance: {final_state.depth}")
print(f"Total number of agents: {len(final_state.agents)}")
print(f"Executation time: {end - start} second")

Number of different states seen: 290955
Number of all states seen: 5270562
Optimal path:
(1,0) 1 UP
(2,0) 1 UP
(2,1) 1 RIGHT
(2,2) 1 RIGHT *boost* number of agents: 2
(5,1) 2 RIGHT
(5,2) 2 RIGHT
(5,3) 2 RIGHT
(4,3) 2 DOWN
(5,3) 2 UP
(5,4) 2 RIGHT
(5,5) 2 RIGHT
(1,2) 1 DOWN
(1,3) 1 RIGHT
(1,4) 1 RIGHT
(1,5) 1 RIGHT
(2,5) 1 UP
(3,5) 1 UP
(4,5) 1 UP
(5,5) 1 UP
Distance: 19
Total number of agents: 2
Executation time: 86.39207196235657 second


## Test result of `IDS` search:

| Test# | Different states  |  All states  | Distance  |             time           |
|------ |-----------------|------------- |-----------|----------------------------|
|   1   |      1893       |    31651     |     11    |     0.3046090602874756     |
|   2   |     166772      |    2619636   |     11    |     29.745219707489014     |
|   3   |     290955      |    5270562   |     19    |     84.30033230781555      |


# Informed Search

# A* (A star) search: minimizing the total estimated solution cost

The `A*` serach algorithm evaluates nodes by combining $g(n)$, the cost to reach the node, and $h(n)$, the cost to getfrom the node to the goal.  $f(n) = g(n) + h(n)$
If we are trying to the cheapest solution, what we should try first is finding the node with the lowest value of $g(n) + h(n)$.

### Measuring `A*` performance:
* **Completeness**: Yes 
* **Optimality**: Yes, A* using `GRAPH-SERACH` is optimal if $h(n)$ in consistent
* **Time Complexity**: number of nodes which heuristic expands
* **Space Complexity**: $O(b^{d})$. 

*`b` is branching factor and `d` is depth of the optimal solution

**Advantages**:

* It is complete and optimal search algorithm.
* It is the best one from other techniques. It is used to solve very complex problems.
* the space complexity is good because of the fact that less nodes are seen(although it is exponential).

**Disadvantages**:
* The speed execution of A* search is highly dependant on the accuracy of the heuristic algorithm that is used to compute $h(n)$.

### Heuristic Function

By the definition, heuristic function is the estimated cost of the cheapest path from node n to a goal node. Here I defined the heuristic function as:

**$h(n) =$ maximum manhattan distance from all agent to the top right corner (final goal node) of the map**

### **This $h(n)$ is Admissible. WHY?**

The definition of `admissible heuristic`: We say `h(n)` is admissible if it never overestimates the cost to reach the goal.

**proof:** In this problem the minimum cost of reaching the goal is when the agent chooses the manhattan path to go to top-right corner of the map and there is no obstacles on this way and also all the potions are in this manhattan path. Hence the $h(n)$ never overestimate the optimal path:

$h(n) \leq g(n) $ which $h(n)$ is the heuristic function and $g(n)$ is actual path cost to goal. 

### **This $h(n)$ is Consistent. WHY?**

By defenition of *Artificial Intelligence(Amodern Approach)* book, a heuristic $h(n)$ is consistent if, for every node $n$ and every successor $n'$ of $n$ generated by any action $a$, te estimated cost of reaching the goal from n is no greater than the step costof getting to $n'$ plus the estimated cost if reaching the goal from $n'$

$h(n) \le c(n,a,n') + h(n')$

This is a form of the general **triangle inequality**, which stipulates that each side of a triangle cannot be longer that the sum of other two sides.

**proof:** conside state `A` and and its successor state `B`. The step cost in this problem is always $1$ so $cost(AtoB) = 1$. By the defined $h(n)$, $h(A) - h(B)$ can be $1$ or $-1$ (since there are just 4 directions available). Hence the inequality mentioned above is always true and we have: $ h(A) - h(B) \leq 1  \rightarrow h(A) - h(B) \leq cost(A to B) \rightarrow$ The heuristic funcion is `Consistent`.


To implement A* algorithm we need priority queue to store frontier states in the order of the `f` function. As I mentioned abobe:  $f(n) = g(n) + h(n)$. In order to do this I implemented the priority queue class which has pop() and push() methods:

In [725]:
class PriorityQueue:
    def __init__(self):
        self.queuee = []
        self.size = 0
        self.indexx = 0
        
    def pop(self):
        self.size -= 1
        return heapq.heappop(self.queuee)[-1]
 
    def push(self, item, priority):
        bisect.insort(self.queuee, (priority, self.indexx, item))
        self.size += 1
        self.indexx += 1

Here I implemented described heuristic function and  $f(n) = g(n) + h(n)$. 

In [726]:
def f(curr_state, alpha):
    cost = curr_state.depth
    min_distance = get_heuristic(curr_state)
    return cost + min_distance

def get_heuristic(curr_state):
    max_dist_agent = -math.inf
    for a in curr_state.agents:
        if abs(m - 1 - a.x) + abs(n - 1 - a.y) > max_dist_agent:
            max_dist_agent = abs(m - 1 - a.x) + abs(n - 1 - a.y)
    return max_dist_agent

This is the A* implementation:

In [729]:
def a_star(initial_state, f_function, alpha):
    number_of_all_states = 0
    frontier = PriorityQueue()
    explored = set()
    explored.add(initial_state.to_string())
    if initial_state.is_goal():
        print("Number of all states seen: ", number_of_all_states)
        return starting_state
    frontier.push(initial_state, f_function(initial_state, alpha))
    while frontier.size:
        curr_state = frontier.pop()
        for d in directions:
            for a in curr_state.agents:
                new_x = a.x + d[0]
                new_y = a.y + d[1]
                if not is_obs(new_x, new_y):
                    new_pos_ag = Agent(new_x, new_y, a.id)
                    new_potions = copy.deepcopy(curr_state.potions)
                    new_boosts = copy.deepcopy(curr_state.boosts)
                    if is_potion_in_sqaure(curr_state, new_x, new_y):
                        new_potions = [x for x in new_potions if x not in [[new_x, new_y]]]
                    new_ag = None
                    if is_boost_in_sqaure(curr_state, new_x, new_y):
                        new_boosts = [x for x in new_boosts if x not in [[new_x, new_y]]]
                        new_ag = Agent(n-1, 0, len(curr_state.agents)+1 )
                    new_state = State(new_pos_ag, new_potions, new_boosts, curr_state)
                    number_of_all_states += 1
                    if not new_ag == None:
                        new_state.add_agent(new_ag)
                    for other_ag in curr_state.agents:
                        if not other_ag.id == new_pos_ag.id:
                            new_state.add_agent(other_ag)
                    move_str = "(" + str(new_pos_ag.x)+","+str(new_pos_ag.y)+") "+str(new_pos_ag.id)+' '+get_dir_string(d)
                    if not new_ag == None:
                        move_str += " *boost* number of agents: " + str(len(new_state.agents))
                    new_state.set_move(move_str)
                    if new_state.to_string() not in explored:
                        if new_state.is_goal():
                            print(f"Different states seen: {len(explored)}")
                            print(f"Number of all states seen: {number_of_all_states}")
                            return new_state
                        frontier.push(new_state, f_function(new_state, alpha))
                        explored.add(new_state.to_string())

In [732]:
start = time.time()
final_state = a_star(initial_state, f, 1)
end = time.time()

Different states seen: 36519
Number of all states seen: 139524


Now I run the A* algorithm:

In [733]:
print("Optimal path:")
print_path(final_state)
print(f"Distance: {final_state.depth}")
print(f"Total number of agents: {len(final_state.agents)}")
print(f"Executation time: {end - start} second")

Optimal path:
(1,0) 1 UP
(2,0) 1 UP
(2,1) 1 RIGHT
(2,2) 1 RIGHT *boost* number of agents: 2
(5,1) 2 RIGHT
(5,2) 2 RIGHT
(5,3) 2 RIGHT
(4,3) 2 DOWN
(1,2) 1 DOWN
(1,3) 1 RIGHT
(1,4) 1 RIGHT
(4,4) 2 RIGHT
(1,5) 1 RIGHT
(2,5) 1 UP
(3,5) 1 UP
(5,4) 2 UP
(4,5) 1 UP
(5,5) 1 UP
(5,5) 2 RIGHT
Distance: 19
Total number of agents: 2
Executation time: 2.9432196617126465 second


## Test result of `A*` search:

| Test# | Different states  |  all states  | Distance  |             time           |
|------ |-----------------|------------- |-----------|----------------------------|
|   1   |      714        |    2853      |     11    |     0.04856371879577637    |
|   2   |     19884       |    64500     |     11    |     1.2474870681762695     |
|   3   |     36519       |    139524    |     19    |     2.962157964706421      |

# Wighted A* (A star) search

The idea of `wighted A* search` is to speed up search at the expense of optimality. It takes an admissible heuristic, `inflate` it by a multiple $\alpha \gt 1$, and then perform A* search.
In this method, fewer nodes tend to get expanded, but the resulting solution may be suboptimal(its cost will be at most $\alpha$ times the cost of the optimal solution)

I chose 2 alphas: $\alpha1 = 1.5$ and $\alpha2 = 2.5$

In [734]:
alpha1 = 1.5
alpha2 = 2.5

def get_weighted_heuristic(curr_state, alpha):
    return alpha * get_heuristic(curr_state)
def get_weighted_f(curr_state, alpha):
    return curr_state.depth + get_weighted_heuristic(curr_state, alpha)

### Result of `weighted A*` with $\alpha = 1.5$:  

In [735]:
start = time.time()
final_state = a_star(initial_state, get_weighted_f, alpha1)
end = time.time()

Different states seen: 10934
Number of all states seen: 32156


In [736]:
print("Optimal path:")
print_path(final_state)
print(f"Distance: {final_state.depth}")
print(f"Total number of agents: {len(final_state.agents)}")
print(f"Executation time: {end - start} second")

Optimal path:
(1,0) 1 UP
(2,0) 1 UP
(2,1) 1 RIGHT
(2,2) 1 RIGHT *boost* number of agents: 2
(1,2) 1 DOWN
(1,3) 1 RIGHT
(1,4) 1 RIGHT
(1,5) 1 RIGHT
(5,1) 2 RIGHT
(5,2) 2 RIGHT
(2,5) 1 UP
(3,5) 1 UP
(5,3) 2 RIGHT
(4,3) 2 DOWN
(4,4) 2 RIGHT
(4,5) 1 UP
(5,4) 2 UP
(5,5) 1 UP
(5,5) 2 RIGHT
Distance: 19
Total number of agents: 2
Executation time: 0.7081477642059326 second


## Test result of `A*` search: ($\alpha = 1.5$)

| Test# | Different states  |  All states | Distance  |             time           |
|------ |-----------------|-------------|-----------|----------------------------|
|   1   |      457        |    1189     |     11    |     0.025171995162963867   |
|   2   |     7364         |    19718     |     11    |     0.029087066650390625   |
|   3   |     10934       |    32156    |     19    |     0.6782541275024414     |

### Result of `weighted A*` with $\alpha = 2.5$:  

In [737]:
start = time.time()
final_state = a_star(initial_state, get_weighted_f, alpha2)
end = time.time()

Different states seen: 4230
Number of all states seen: 11288


In [738]:
print("Optimal path:")
print_path(final_state)
print(f"Distance: {final_state.depth}")
print(f"Total number of agents: {len(final_state.agents)}")
print(f"Executation time: {end - start} second")

Optimal path:
(1,0) 1 UP
(2,0) 1 UP
(2,1) 1 RIGHT
(2,2) 1 RIGHT *boost* number of agents: 2
(1,2) 1 DOWN
(1,3) 1 RIGHT
(1,4) 1 RIGHT
(1,5) 1 RIGHT
(5,1) 2 RIGHT
(2,5) 1 UP
(5,2) 2 RIGHT
(3,5) 1 UP
(5,3) 2 RIGHT
(4,3) 2 DOWN
(5,3) 2 UP
(5,4) 2 RIGHT
(4,5) 1 UP
(5,5) 1 UP
(5,5) 2 RIGHT
Distance: 19
Total number of agents: 2
Executation time: 0.2266552448272705 second


## Test result of `A*` search: ($\alpha = 2.5$)

| Test# | unique  states  |  all states  | Distance  |             time           |
|------ |-----------------|------------- |-----------|----------------------------|
|   1   |      312        |    786       |     11    |     0.013810396194458008   |
|   2   |     1509         |    3032       |     11    |     0.021607160568237305   |
|   3   |     4230        |    11288     |     19    |     0.23101449012756348    |

As we see above, by increasing the $\alpha$ value, the states were seen and as a result the excutation time decreased. here the optimal path did not change but in some problems it is possible to not to get the optimal solution.

## Comapring each test by each algorithm:

### Test#1

|       Algorithm               | Different states|  All states  | Distance  |            average time    |
|------------------------------|-----------------|------------- |-----------|----------------------------|
|   BFS                        |     805         |    4149       |     11    |     0.05683951914572177    |
|   IDS                        |     1893        |    31651       |     11    |     0.30469679832458496    |
|   A*                         |     714         |    2853      |     11    |     0.04619717597961426    |
|   weighted A* ($\alpha=1.5$) |     457         |    1189      |     11    |     0.026871204376220703   |
|   Weighted A* ($\alpha=2.5$) |     312         |    786       |     11    |     0.01796889305114746    |

### Test#2

|           Algorithm            | Different states|  All states  | Distance  |            average time    |
|------------------------------|-----------------|------------- |-----------|----------------------------|
|   BFS                        |      854407     |    116268    |     11    |     10.246020793914795   |
|   IDS                        |     166772      |    2619636   |     11    |     29.92528986930847     |
|   A*                         |     19884       |    64500     |     11    |     1.0177013874053955    |
|   weighted A* ($\alpha=1.5$) |     7364        |    11288     |     11    |      0.31426143646240234    |
|   Weighted A* ($\alpha=2.5$) |     1509        |    3032     |     11    |     0.06974935531616211    |

### Test#3

|         Algorithm         | Different states|  All states  | Distance  |            average time    |
|------------------------------|-----------------|------------- |-----------|----------------------------|
|   BFS                        |      179473     |    1082141   |     19    |    15.309660911560059      |
|   IDS                        |     290955      |    5270562    |     19    |     82.06716203689575   |
|   A*                         |     36519        |    139524     |     19    |     2.92076420783996648    |
|   weighted A* ($\alpha=1.5$) |     10934        |    32156     |     19    |     0.6861546039581299    |
|   Weighted A* ($\alpha=2.5$) |     4230        |    11288     |     19    |     0.22646188735961914    |

## Comparing and Concolusion

As we can see above all these algorithms are optimal and complete. They found the optimal solution but there are some differences which we should consider based on our aim. sometimes time is important sometime it is not.
Generally, Informed search algorithms are better than Uninformed search algorithms becuase the have "brain". Informed serach algorithms are faster that Uninformed ones. They estimate the cost and decide in each state which path is better according to the estimatin.
when we want to find the solution faster with less space complexiry and using Uninformed algorithms, it is better to choose `BFS` rather than `IDS`, otherwise we can choose `IDS`. In tests above, the executation time of `IDS` was more than `BFS`. When we want to use Informed Search algorithms and we want the optimal solution, the best choice is `A*` but if we want to trade off the time with optimality we can choose `weighted A*`. In `weighted A*` we get to the solution(but maybe not the optimal solution) even faster than `A*`. Also, if we increase the weight the accuracy would get lower but the executation time will increase and also there is more likely not to get the optimal solution.

In this project I learned about different search algorithms, how to implement them and their advantages and disadvantages. Now I can compare them and finally choose which algorithms above are better in different problems.